In [1]:
import pandas as pd 
import numpy as np
from lightfm import LightFM, cross_validation
from lightfm.evaluation import precision_at_k, auc_score
from lightfm.data import Dataset
import pickle 




C:\ProgramData\Anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
df1 = pd.read_csv('ratings.csv')
df2 = pd.read_csv('movies.csv')
rating_df = df2.merge(df1, on = 'movieId' ).dropna()
df = rating_df[['userId','movieId','rating']]
df4 = rating_df[['userId','movieId','rating','title','genres']]


# Model 1 

In [10]:
data = Dataset()
data.fit(df.userId.unique(), df.movieId.unique())

interactions_matrix, weights_matrix = data.build_interactions([tuple(i) for i in df.values])

train, test = cross_validation.random_train_test_split(interactions_matrix, test_percentage=0.25, random_state=2022)

train_weight , test_weight = cross_validation.random_train_test_split(weights_matrix, test_percentage=0.25, random_state=2022)



model_wrap = LightFM(loss='warp', learning_rate=0.01, k=10)
model_wrap.fit(train, sample_weight = train_weight , epochs=150)

train_precision = precision_at_k(model_wrap, train, k=10).mean()
test_precision = precision_at_k(model_wrap, test, k=10, train_interactions=train).mean()

train_auc = auc_score(model_wrap, train).mean()
test_auc = auc_score(model_wrap, test,train_interactions=train).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.38, test 0.23.
AUC: train 0.94, test 0.92.


In [11]:
pickle.dump(model_wrap , open('model1.pkl','wb'))

### Model 2

In [3]:
uf = [] 
col = ['movieId']*len(df4['movieId'].unique()) + ['rating']*len(df4['rating'].unique()) + ['title']*len(df4['title'].unique()) + ['genres']*len(df4['genres'].unique()) 
unique_f1 = list(df4['movieId'].unique()) + list(df4['rating'].unique()) + list(df4['title'].unique()) + list(df4['genres'].unique()) 
#print('f1:', unique_f1)
for x,y in zip(col, unique_f1):
    res = str( x)+ ":" +str(y)
    uf.append(res)
    #print(res)
                   

In [4]:
data = Dataset()
#data.fit(df.userId.unique(), df.movieId.unique())
data.fit( 
        df4['userId'].unique(), 
        df4.movieId.unique(), # tous les éléments
        user_features = uf # fonctionnalités utilisateur supplémentaires
 )

In [5]:
interactions_matrix, weights_matrix = data.build_interactions([tuple(i) for i in df.values])

In [6]:
# Helper function that takes the user features and converts them into the proper "feature:value" format
def feature_colon_value(my_list):
    """
    Takes as input a list and prepends the columns names to respective values in the list.
    For example: if my_list = [1,1,0,'del'],
    resultant output = ['movieId','rating','title','genres']
   
    """
    result = []
    ll = ['movieId:','rating:','title:','genres:']
    aa = my_list
    for x,y in zip(ll,aa):
        res = str(x) +""+ str(y)
        result.append(res)
    return result
# Using the helper function to generate user features in proper format for ALL users
ad_subset = df4[['movieId','rating','title','genres']] 
ad_list = [list(x) for x in ad_subset.values]
feature_list = []
for item in ad_list:
    feature_list.append(feature_colon_value(item))
#print(f'Final output: {feature_list}')

In [7]:
user_tuple = list(zip(df['userId'], feature_list))
user_features = data.build_user_features(user_tuple, normalize= False)
user_id_map, user_feature_map, movie_id_map, movie_feature_map = data.mapping()

In [9]:
user_id_map

{7: 0,
 9: 1,
 13: 2,
 15: 3,
 19: 4,
 20: 5,
 23: 6,
 26: 7,
 30: 8,
 37: 9,
 43: 10,
 44: 11,
 47: 12,
 48: 13,
 55: 14,
 56: 15,
 63: 16,
 67: 17,
 68: 18,
 69: 19,
 70: 20,
 72: 21,
 73: 22,
 75: 23,
 77: 24,
 79: 25,
 84: 26,
 86: 27,
 87: 28,
 89: 29,
 90: 30,
 91: 31,
 92: 32,
 93: 33,
 94: 34,
 97: 35,
 99: 36,
 100: 37,
 106: 38,
 112: 39,
 119: 40,
 120: 41,
 121: 42,
 122: 43,
 124: 44,
 125: 45,
 126: 46,
 128: 47,
 130: 48,
 134: 49,
 136: 50,
 138: 51,
 142: 52,
 146: 53,
 149: 54,
 150: 55,
 152: 56,
 153: 57,
 154: 58,
 157: 59,
 163: 60,
 164: 61,
 165: 62,
 168: 63,
 169: 64,
 173: 65,
 175: 66,
 176: 67,
 177: 68,
 178: 69,
 179: 70,
 184: 71,
 185: 72,
 187: 73,
 193: 74,
 200: 75,
 201: 76,
 205: 77,
 212: 78,
 213: 79,
 215: 80,
 219: 81,
 220: 82,
 224: 83,
 232: 84,
 237: 85,
 240: 86,
 241: 87,
 242: 88,
 247: 89,
 252: 90,
 253: 91,
 261: 92,
 262: 93,
 268: 94,
 272: 95,
 273: 96,
 275: 97,
 282: 98,
 283: 99,
 284: 100,
 285: 101,
 287: 102,
 292: 103,
 293:

In [8]:
model = LightFM(loss='warp')
model.fit(interactions_matrix,
      user_features= user_features,
      sample_weight= weights_matrix,
      epochs=1)

KeyboardInterrupt: 

In [ ]:
train_auc = auc_score(model,
                      interactions_matrix,
                      user_features=user_features
                     ).mean()
print('Hybrid training set AUC: %s' % train_auc)


In [38]:
# predict for existing user
user_x = user_id_map[7]
n_users, n_items = interactions_matrix.shape # no of users * no of items
model.predict(user_x, np.arange(n_items)) # means predict for all 

array([-1.3677758e+07, -2.7999306e+08,  2.8582781e+09, ...,
       -3.5660500e+09,  9.0468291e+09,  1.2076840e+09], dtype=float32)

In [39]:
from scipy import sparse
def format_newuser_input(user_feature_map, user_feature_list):
    num_features = len(user_feature_list)
    normalised_val = 1.0 
    target_indices = []
    for feature in user_feature_list:
        try:
            target_indices.append(user_feature_map[feature])
        except KeyError:
            print("new user feature encountered '{}'".format(feature))
            pass

    new_user_features = np.zeros(len(user_feature_map.keys()))
    for i in target_indices:
        new_user_features[i] = normalised_val
        new_user_features = sparse.csr_matrix(new_user_features)
    return(new_user_features)

In [40]:
def recommend(model, user_id):
    n_users, n_items = train.shape
    user_id_map, user_feature_map, movie_id_map, movie_feature_map = data.mapping()

    user_feature_list = ['movieId:1', 'rating:1', 'title:Toy Story (1995)', 'genres:Adventure|Animation|Children|Comedy|Fantasy']
    new_user_features = format_newuser_input(user_feature_map, user_feature_list)


    best_rated = rating_df[(rating_df.userId == user_id) & (rating_df.rating >= 4.5)].movieId.values
    if best_rated.shape[0] == 0 :
        scores = model.predict(0, np.arange(n_items), user_features=new_user_features) 
        top_items = rating_df['title'][np.argsort(-scores)]
        print("\nRecommended:")
        for x in top_items[:10]:
            print(x)
        
        return list(top_items)

    else : 
        known_positives = rating_df.loc[rating_df['movieId'].isin(best_rated)].title.values
        scores = model.predict(user_id, np.arange(n_items)) 
        top_items = rating_df['title'][np.argsort(-scores)]
        ls = []
        for x in known_positives:
            if x not in ls :
                ls.append(x)
                
        print("User %s likes:" % user_id)
        for k in ls:
            print(k)
            
        print("\nRecommended:")
        for x in top_items[:10]:
            print(x)
    
        return list(ls,top_items)

In [46]:
user_x = user_id_map[7]

recommend(model,1)

new user feature encountered 'rating:1'


IndexError: row index (9747) out of range

In [ ]:

#pickle.dump(model , open('model.pkl','wb'))

In [ ]:
#model_exp = pickle.load(open("model.pkl",'rb'))

In [43]:
user_x = user_id_map[7]
n_users, n_items = interactions_matrix.shape # no of users * no of items
model.predict(user_x, np.arange(n_items)) # means predict for all 

array([-1.3677758e+07, -2.7999306e+08,  2.8582781e+09, ...,
       -3.5660500e+09,  9.0468291e+09,  1.2076840e+09], dtype=float32)